## Build Functions for ELT

In [1]:
# Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 uninstall pandas pyarrow fireducks numpy -y
# !pip3 install -r requirements.txt

In [2]:

import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime
import pytz

import pprint

from src import modules as f # f is for function
%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [3]:
# manufactoring suppliers 
# 'lrcs', 'klac', ''

In [11]:
f.predictions('amd')

AMD 5m Interval Timestamp: 2024-12-09 02:35:35 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,static
kelly_1:2.5,0.28065,0.301188,0.29182,0.194003
prob_up,0.21553,0.264387,0.288361,0.375
prob_static,0.630123,0.516926,0.461806,0.375
prob_down,0.154347,0.218687,0.249833,0.25
precision,0.486179,0.500849,0.494157,0.424288
recall,0.613963,0.605749,0.607803,0.581109
f1,0.54265,0.548327,0.54512,0.490468
support,"[487.0, 487.0, 488.0]","[487.0, 487.0, 488.0]","[487.0, 487.0, 488.0]","[487.0, 487.0, 488.0]"


AMD 15m Interval Timestamp: 2024-12-09 02:35:38 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,up,up,static
kelly_1:2.5,0.271345,0.134545,0.188,0.19207
prob_up,0.32542,0.40094,0.372018,0.125
prob_static,0.457589,0.325101,0.347374,0.75
prob_down,0.216991,0.273959,0.280608,0.125
precision,0.479532,0.381818,0.42,0.422907
recall,0.503067,0.388889,0.388889,0.588957
f1,0.491018,0.385321,0.403846,0.492308
support,"[162.0, 163.0, 161.0]","[162.0, 163.0, 161.0]","[162.0, 163.0, 161.0]","[162.0, 163.0, 161.0]"


AMD 1h Interval Timestamp: 2024-12-09 02:35:40 


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,down,down
kelly_1:2.5,0.10202,0.080242,0.056566,0.00036
prob_up,0.288927,0.304012,0.309961,0.25
prob_static,0.355573,0.349348,0.342757,0.25
prob_down,0.3555,0.34664,0.347282,0.5
precision,0.358586,0.34303,0.326118,0.285971
recall,0.378667,0.377333,0.302139,0.212567
f1,0.368353,0.359365,0.313671,0.243865
support,"[749.0, 750.0, 748.0]","[749.0, 750.0, 748.0]","[749.0, 750.0, 748.0]","[749.0, 750.0, 748.0]"


AMD 1wk Interval Timestamp: 2024-12-09 02:35:47 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,down,down,up
kelly_1:2.5,0.165753,0.091892,0.140506,0.076
prob_up,0.283046,0.313075,0.321124,0.375
prob_static,0.329488,0.340383,0.312516,0.25
prob_down,0.387466,0.346542,0.36636,0.375
precision,0.40411,0.351351,0.386076,0.34
recall,0.383117,0.337662,0.396104,0.333333
f1,0.393333,0.344371,0.391026,0.336634
support,"[153.0, 153.0, 154.0]","[153.0, 153.0, 154.0]","[153.0, 153.0, 154.0]","[153.0, 153.0, 154.0]"


AMD 1mo Interval Timestamp: 2024-12-09 02:35:49 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,up,up
kelly_1:2.5,0.152632,0.35,0.174359,0.097778
prob_up,0.903064,0.87702,0.673754,0.75
prob_static,0.075493,0.092018,0.210204,0.25
prob_down,0.021443,0.030962,0.116041,0.0
precision,0.394737,0.535714,0.410256,0.355556
recall,0.5,0.5,0.533333,0.533333
f1,0.441176,0.517241,0.463768,0.426667
support,"[30.0, 30.0, 30.0]","[30.0, 30.0, 30.0]","[30.0, 30.0, 30.0]","[30.0, 30.0, 30.0]"


# 

In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')